In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df = pd.read_pickle("/content/drive/My Drive/data_cleaned_pkl", compression="gzip")
df = df.reset_index()
df = df.drop("index", axis=1)
df = df.drop("LYRICS_REGION", axis=1)

In [0]:
df

,ARTIST_NAME,ARTIST_URL,SONG_NAME,SONG_URL,LYRICS,GENRE
0,03 greedo,https://www.azlyrics.com/19/03greedo.html,sweet lady,https://www.azlyrics.com/lyrics/03greedo/sweet...,"its only one, 03, i'm from grape street, where...",urban
1,03 greedo,https://www.azlyrics.com/19/03greedo.html,mafia business,https://www.azlyrics.com/lyrics/03greedo/mafia...,"you gonna make me put you in a suit and tie, s...",urban
2,03 greedo,https://www.azlyrics.com/19/03greedo.html,paranoid,https://www.azlyrics.com/lyrics/03greedo/paran...,"we could kill it, yeah, we could, we could, we...",urban
3,03 greedo,https://www.azlyrics.com/19/03greedo.html,never bend,https://www.azlyrics.com/lyrics/03greedo/never...,"yeah, you lil bitch ass dudes steady speaking ...",urban
4,03 greedo,https://www.azlyrics.com/19/03greedo.html,prayer for my lost,https://www.azlyrics.com/lyrics/03greedo/praye...,"x loaded up bro, never see me in the church, p...",urban
...,...,...,...,...,...,...
73552,zug izland,https://www.azlyrics.com/z/zugizland.html,cry,https://www.azlyrics.com/lyrics/zugizland/cry....,"our fire will rise, ringmaster's eyes, funhous...",urban
73553,zug izland,https://www.azlyrics.com/z/zugizland.html,suicide,https://www.azlyrics.com/lyrics/zugizland/suic...,"hey there killer, where you goin'?, i see you ...",urban
73554,zug izland,https://www.azlyrics.com/z/zugizland.html,i found you,https://www.azlyrics.com/lyrics/zugizland/ifou...,"life was shitty, can't call it, my karmas, bee...",urban
73555,zug izland,https://www.azlyrics.com/z/zugizland.html,twilight zone,https://www.azlyrics.com/lyrics/zugizland/twil...,"somewhere in a lonely hotel room, there's a gu...",urban


In [3]:
!pip install pytorch_pretrained_bert
!pip install pronouncing

     |████████████████████████████████| 133kB 3.4MB/s 
     |████████████████████████████████| 942kB 5.2MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6223 sha256=00dcc65f2e6d2fab728af6c650216f92fed1638fcc11b3b8485c8be26c000815
  Stored in directory: /root/.cache/pip/wheels/81/fd/e8/fb1a226f707c7e20dbed4c43f81b819d279ffd3b0e2f06ee13
Successfully built pronouncing


In [4]:
import itertools
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForNextSentencePrediction
import pronouncing
import re

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-large-uncased')
model.eval()

sentence = '[CLS] Charles is a tailor [SEP] He is tall [SEP]'
# sentence = '[CLS] Charles is a tailor [SEP] Excavation is important [SEP]'
toks = tokenizer.tokenize(sentence)
tok_ids = tokenizer.convert_tokens_to_ids(toks)
tok_tensor = torch.LongTensor([tok_ids])
token_type_ids_tensor = torch.LongTensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])
seq_relationship_logits = model(tok_tensor, token_type_ids_tensor)
print(seq_relationship_logits)

100%|██████████| 1248501532/1248501532 [00:21<00:00, 58659142.41B/s]


tensor([[ 6.8790, -5.0848]], grad_fn=<AddmmBackward>)


In [0]:
def predict_next_sentence(sentenceA, sentenceBs, tokenizer, model, rhyme=False):
    seq_relationship_logits = get_next_sentence_logits(sentenceA, sentenceBs, tokenizer, model)
    if rhyme:
        for i, sentenceB in enumerate(sentenceBs):
            last_word_A = re.sub(r'[^\w]', '', sentenceA.split()[-1])
            last_word_B = re.sub(r'[^\w]', '', sentenceB.split()[-1])
            if last_word_A in pronouncing.rhymes(last_word_B) or \
                last_word_B in pronouncing.rhymes(last_word_A):
                print(f'{last_word_A} rhymes with {last_word_B}')
                seq_relationship_logits[i, 0] += 10
    return sentenceBs[seq_relationship_logits[:,0].argmax().tolist()]

def get_ids_types_attention_from_sentence_pair(sentenceA, sentenceB, pad_total_size, tokenizer):
    sentenceA_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentenceA))
    sentenceB_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentenceB))
    padding_size = pad_total_size - len(sentenceA_ids) - len(sentenceB_ids)
    sentence_ids = sentenceA_ids + sentenceB_ids + [0] * padding_size
    sentence_types = [0] * len(sentenceA_ids) + [1] * len(sentenceB_ids) + [0] * padding_size
    sentence_attention = [1] * (len(sentenceA_ids) + len(sentenceB_ids)) + [0] * padding_size
    return sentence_ids, sentence_types, sentence_attention

def reconstruct_song(lines, tokenizer, model):
    pad_total_size = 2 * max(len(line) for line in lines)
    for ordering in itertools.permutations(lines):
        pair_ids, pair_types, pair_attentions = [], [], []        
        for i in range(len(ordering) - 1):
            pair_id, pair_type, pair_attention = \
                get_ids_types_attention_from_sentence_pair(ordering[i], ordering[i + 1], pad_total_size, tokenizer)
            pair_ids.append(pair_id)
            pair_types.append(pair_type)
            pair_attentions.append(pair_attention)
        ids_tensor = torch.LongTensor(pair_ids)
        types_tensor = torch.LongTensor(pair_types)
        attention_tensor = torch.LongTensor(pair_attentions)
        seq_relationship_logits = model(ids_tensor, types_tensor, attention_tensor)
#         print(seq_relationship_logits)
#         print(sum(seq_relationship_logits[:, 0].tolist()))
#         print(ordering)
    
    
def get_next_sentence_logits(sentenceA, sentenceBs, tokenizer, model):
    sentenceA_toks = tokenizer.tokenize(sentenceA)
    sentenceA_ids = tokenizer.convert_tokens_to_ids(sentenceA_toks)
    sentenceA_types = [0] * len(sentenceA_ids)
    sentenceA_attention = [1] * len(sentenceA_ids)
    tok_ids = []
    tok_types = []
    tok_attention = []
    
    sentenceBs_ids = []
    for sentenceB in sentenceBs:
        sentenceB_toks = tokenizer.tokenize(sentenceB)
        sentenceB_ids = tokenizer.convert_tokens_to_ids(sentenceB_toks)
        sentenceBs_ids.append(sentenceB_ids)
        
    max_sentenceB_length = max(len(sentenceB_ids) for sentenceB_ids in sentenceBs_ids)
    for sentenceB_ids in sentenceBs_ids:
        padding_size = max_sentenceB_length - len(sentenceB_ids)
        padded_sentenceB_ids = sentenceB_ids + [0] * padding_size
        padded_sentenceB_types = [1] * max_sentenceB_length
        padded_sentenceB_attention = [1] * len(sentenceB_ids) + [0] * padding_size
        tok_ids.append(sentenceA_ids + padded_sentenceB_ids)
        tok_types.append(sentenceA_types + padded_sentenceB_types)
        tok_attention.append(sentenceA_attention + padded_sentenceB_attention)
    tok_ids_tensor = torch.LongTensor(tok_ids)
    tok_types_tensor = torch.LongTensor(tok_types)
    tok_attention_tensor = torch.LongTensor(tok_attention)
    seq_relationship_logits = model(tok_ids_tensor, tok_types_tensor, tok_attention_tensor)
    return seq_relationship_logits

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-large-uncased')
model.eval()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace

In [0]:
import csv
import random            
    
def generate_song():
    num_lines = 10
    num_choices = 100
    
    all_lines = []
    genre = input("Please input genre(pop, urban or rock)\n")
    song_lines = [input("Please input the first line\n")]
    for index, row in df.iterrows():
        if (row['GENRE' == genre]):
          lines = row["LYRICS"].split(',')
          for i in range(len(lines)):
              if lines[i] == '' or lines[i] == ' ':
                pass
              else:
                all_lines.append(lines[i])
    
    
    
    for i in range(num_lines - 1):
        lines = random.sample(all_lines, num_choices)
        next_line = predict_next_sentence(song_lines[-1], lines, tokenizer, model, rhyme=True)
        song_lines.append(next_line)

    with open('generate_song.txt', 'w') as file_song:
        for line in song_lines:
            file_song.write(f'{line}\n')

In [15]:
generate_song()

Please input genre(pop, urban or rock)
pop
Please input the first line
i was walking down the street
flow rhymes with oh
flow rhymes with oh
oh rhymes with ago
oh rhymes with go
ago rhymes with low
ago rhymes with oh
ago rhymes with know
ago rhymes with oh
know rhymes with go
know rhymes with go
know rhymes with no
know rhymes with go
know rhymes with whoa
go rhymes with know
know rhymes with oh
know rhymes with so
know rhymes with no
know rhymes with go
go rhymes with know
go rhymes with know
go rhymes with oh
go rhymes with bro
go rhymes with know
bro rhymes with go
bro rhymes with know
